<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/clustering/image_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project we want using clustering algorithms, analysing flowers pictures an d then based on our data put them into different clusters

# Imports And Settings

## Imports

In [16]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Tuple
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from dataclasses import dataclass

## Settings

In [4]:
DATASET_PATH=os.path.join(os.getcwd(), 'drive/MyDrive/AI-UT/clustering/data')
FLOWER_IMAGES_PATH=os.path.join(DATASET_PATH, 'flower_images/')

IMG_HEIGHT=224
IMG_WIDTH=224

## Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing and Feature Extraction
In this section, we are going to extract information from images using VGG16.

1. Why do we need feature extraction? why dont we only read raw pixel? explain it.
> Instead of using the raw pixels directly, features represent more abstract and meaningful characteristics, such as edges, textures, or shapes.<br>
> Advantages of feature extraction include:<br>
> - **Dimensionality Reduction:** Extracted features often have lower dimensionality than raw pixel data, reducing the complexity of the data and computation.
> -**Increased Robustness:** Features focus on important aspects, making models more robust to variations like changes in lighting, rotation, or scale.
> - **Improved Generalization:** Features highlight relevant information, helping models generalize better to new, unseen data by emphasizing key patterns.
> - **Enhanced Interpretability:** Features can provide insights into the image content, aiding in the interpretability of the model's decisions.
> - **Computational Efficiency:** Working with extracted features is computationally more efficient than processing raw pixel values, enabling faster training and inference. <br>
> - Overall, feature extraction enhances the performance and interpretability of image processing models by transforming raw pixel data into a more meaningful and manageable representation.

2. Explain three ways of feature extraction Theqniques from images.
>- **Convolutional Neural Networks(CNN)**: CNNs are generally the preferred choice for feature extraction from images because CNNs are specifically designed for processing color images and perform more complex tasks such as image classification, object detection, or segmentation where it can extract complex and descriptive features with any variations such as lighting conditions, scale, and other factors in the image. It is useful in situations where you want to have high accuracy after processing your images, thus making it the most efficient way.<br>
>- **Mean pixel values of channels**: It is a very basic image feature extraction technique where we calculate the mean intensity of each color channel (red, green, and blue) in an image. This approach can be useful in situations where the goal is to perform a simple image processing operation or to obtain a basic representation of the image such as thumbnail creation, image compression and resizing. However, mean pixel value of channels is generally not a suitable technique for more complex tasks such as image classification, object detection, or segmentation. This is because mean pixel value of channels does not take into account the spatial relationship between the pixels in the image, and it does not consider the relationship between different channels.<br>
>- **Local Binary Patterns (LBP)**: Local Binary Patterns (LBP) is a widely used feature extraction method for analyzing texture information in images. LBP captures the local structure of an image by comparing the gray values of a pixel to its surrounding neighbors and encoding the result into a binary pattern. LBP features are often used where there is limited data and in machine learning algorithms for various computer vision tasks such as face recognition, texture analysis, and object recognition. LBP is computationally efficient and can operate in real-time applications, such as surveillance and tracking systems, where the processing speed is crucial.

3. Which preprocessing should be applied to image data to prepare it for input into a model?
> There are several techniques used in image preprocessing:
> - Resizing: Resizing images to a uniform size is important for machine learning algorithms to function properly.
> - Grayscaling: Converting color images to grayscale can simplify your image data and reduce computational needs for some algorithms.
> - Noise reduction: Smoothing, blurring, and filtering techniques can be applied to remove unwanted noise from images.
> - Normalization: Normalization adjusts the intensity values of pixels to a desired range, often between 0 to 1. This can improve the performance of machine learning models.
> - Binarization: Binarization converts grayscale images to black and white by thresholding.
> - Contrast enhancement: The contrast of images can be adjusted using histogram equalization.

## Load Dataset
First Let's read our image dataset

In [7]:
@dataclass
class Flower:
  image: np.ndarray
  label: str
  features: np.ndarray

In [8]:
def load_images() -> List[Flower]:
  flowers = []
  labels = pd.read_csv(os.path.join(DATASET_PATH, 'flower_labels.csv'))
  for flower_image in os.listdir(FLOWER_IMAGES_PATH):
    try:
      image = cv2.imread(os.path.join(FLOWER_IMAGES_PATH, flower_image))
      if image is None:
        print(f"failed to read {flower_image}.")
        continue
      label = labels.loc[labels.file == flower_image, 'label'].iloc[0]
      flowers.append(Flower(image=cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT)),
                            label=label,
                            features=None))
    except Exception as e:
      print(f"{str(e)}")

  return flowers

In [9]:
flowers = load_images()

## Feature Extraction


First, let's create our VGG16 model

In [23]:
vgg16 = VGG16(weights='imagenet',
              include_top=False,
              input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Next, using VGG16, we are going to extract features from our images.

In [27]:
def extract_features(flowers: List[Flower], model: VGG16=vgg16) -> None:
  feat_extractor = Model(inputs=model.input, outputs=model.output)
  for flower in flowers:
    img = flower.image
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    flower.features = model.predict(img)

In [28]:
extract_features(flowers)

1/1 [==============================] - 1s 536ms/step
